.ipynb settings

In [29]:
#Settings relative to .ipynb file format that have to be executed here

#Make reloading of modules automatic et évite d'avoir à redémarer le kernel et tout re executer pour appliquer la modification d'un module dans __main__.
%load_ext autoreload  
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
#src.useful_functions
import sys
sys.path.append('../src')  # Adjust the path to import modules in src/

import useful_functions

Importation of the dataset(s)

In [31]:
#Importing dataset(s)
from data import make_dataset

#Load the feat_engineered_ds:
X_train_info = make_dataset.load_data(True, 'processed/X_train_info')
X_train_00   = make_dataset.load_data(False, 'processed/X_train_00')
Y_train_00   = make_dataset.load_data(False, 'processed/Y_train_00')
X_test_info  = make_dataset.load_data(True, 'processed/X_test_info')
X_test_00    = make_dataset.load_data(False, 'processed/X_test_00')
Y_test_00    = make_dataset.load_data(False, 'processed/Y_test_00')


The processed/X_train_info dataframe contains matchs of the seasons:  [2015 2016 2017 2018 2019 2020]
The processed/X_test_info dataframe contains matchs of the seasons:  [2021 2022 2023]


# VI) <u> Pipeline development 

<span style="color:#FF1493;"> <strong> <font size="4">1) Pipeline and GridSearchCV implementation

In [32]:

# --------------------------------------------------------------
# Correlated features remover (not used)
# --------------------------------------------------------------

from features import features_selection

"""
#Correlated features remover definition (it's a transformer)
corr_features_selector = features_selection.correlated_features_removal_transformer(corr_threshold = 0.95)


# --------------------------------------------------------------
# Features selector
# --------------------------------------------------------------

"""
from sklearn.linear_model import LogisticRegression
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

model_sfs = LogisticRegression(max_iter=10000, C = 0.3, random_state =33, penalty = 'l2')
sequ_feat_selector = SFS(estimator = model_sfs, k_features = 'best', forward = True, verbose = 0, cv=constant_variables.CV, scoring = 'neg_log_loss', n_jobs = -1)
"""

#OU

#Filter features selection
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
filter_feat_selector = SelectKBest(f_classif, k= 10)

# --------------------------------------------------------------
# Scaler
# --------------------------------------------------------------

from sklearn.preprocessing import RobustScaler, StandardScaler
scaler = StandardScaler()


# --------------------------------------------------------------
# Model
# --------------------------------------------------------------
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty = 'l1', fit_intercept=True, random_state = 999, solver = 'saga', max_iter= 3000, verbose = False, tol=1e-4)

# --------------------------------------------------------------
# Pipeline
# --------------------------------------------------------------
from sklearn.pipeline import Pipeline
pipeline = Pipeline(steps=[("scaler",scaler), ("features_selector",filter_feat_selector), ("model", model)])


#parameters to optimize definition
import numpy as np
parameters = {'model__penalty': ['l1', 'l2', 'elasticnet', 'None'],
             'model__C': np.logspace(-3, 1, 21),
             'model__solver': ['saga', 'sag', 'liblinear', 'newton-cg', 'lbfgs'],
             'features_selector__k' : [5,8,10,11,12,14,15,16,17]
             }

# --------------------------------------------------------------
# GridSearchCV
# --------------------------------------------------------------
# Create a GridSearchCV object
from sklearn.model_selection import GridSearchCV
from configuration import constant_variables

grid_search = GridSearchCV(pipeline, parameters, cv=constant_variables.CV, scoring = 'neg_log_loss', verbose=0)
# Fit the GridSearchCV to the data
grid_search.fit(X_train_00.copy(), Y_train_00.copy().values.ravel())
"""



'\n\n#OU\n\n#Filter features selection\nfrom sklearn.feature_selection import SelectKBest\nfrom sklearn.feature_selection import f_classif\nfilter_feat_selector = SelectKBest(f_classif, k= 10)\n\n# --------------------------------------------------------------\n# Scaler\n# --------------------------------------------------------------\n\nfrom sklearn.preprocessing import RobustScaler, StandardScaler\nscaler = StandardScaler()\n\n\n# --------------------------------------------------------------\n# Model\n# --------------------------------------------------------------\nfrom sklearn.linear_model import LogisticRegression\nmodel = LogisticRegression(penalty = \'l1\', fit_intercept=True, random_state = 999, solver = \'saga\', max_iter= 3000, verbose = False, tol=1e-4)\n\n# --------------------------------------------------------------\n# Pipeline\n# --------------------------------------------------------------\nfrom sklearn.pipeline import Pipeline\npipeline = Pipeline(steps=[("scaler",s

<span style="color:#FF1493;"> <strong> <font size="4">2) Display GridSearchCV results

In [33]:
#src.pipeline.results

from pipeline import results
"""
#We print the results of GridSearchCV() execution
results.GridSearchCV_results(grid_search, X_train_00.copy())

"""

'\n#We print the results of GridSearchCV() execution\nresults.GridSearchCV_results(grid_search, X_train_00.copy())\n\n'

<span style="color:#FF1493;"> <strong> <font size="4">3) Model, features, scaler selection


In [38]:
#src.pipeline.model
from pipeline import model
from features.features_selection import restricted_datasets


"""
#On récupère la pipeline avec ses parametres optimaux 
chosen_pipeline = grid_search.best_estimator_
print(chosen_pipeline)

X_train_01 = X_train_00.copy()
X_test_01 = X_test_00.copy()

chosen_pipeline_trained = chosen_pipeline.fit(X_train_01.copy(), Y_train_00.copy().values.ravel())
"""
#On choisit une pipeline enregistrée dans pipeline.model et la selection de features qui va avec

chosen_pipeline = model.pipeline_03
chosen_features = model.features_selected_03

X_train_01 = X_train_00.copy()[chosen_features]

X_test_01 = X_test_00.copy()[chosen_features]

# Train the pipeline chosen
chosen_pipeline_trained = chosen_pipeline.fit(X_train_01.copy(), Y_train_00.copy().values.ravel())

#Save the trained pipeline

model.save_pipeline(chosen_pipeline_trained, "pipeline_03_trained")


Successfully deleted the old pipeline:     pipeline_03_trained
Successfully saved the new pipeline:       pipeline_03_trained




<span style="color:#808080;"> <strong> <font size="4">4) Model calibration (Not used anymore)

In [35]:

#definition of a calibrator
"""
from sklearn.calibration import CalibratedClassifierCV
calibrated_pipeline = CalibratedClassifierCV(chosen_pipeline_trained, cv = 'prefit' , method = 'isotonic', ensemble = True)
calibrated_pipeline.fit(X_valid_00.copy(), Y_valid_00.copy())
"""


"\nfrom sklearn.calibration import CalibratedClassifierCV\ncalibrated_pipeline = CalibratedClassifierCV(chosen_pipeline_trained, cv = 'prefit' , method = 'isotonic', ensemble = True)\ncalibrated_pipeline.fit(X_valid_00.copy(), Y_valid_00.copy())\n"